In [2]:
import pandas as pd
import json
import os

In [6]:
total_folder = 'Nuggets/!season_totals'
game_dir = 'data/12.06.23_Clippers'

for subdir, dirs, files in os.walk(game_dir):
    for file in files:
        if subdir == r'data/12.06.23_Clippers\Nuggets\Defense':
            overall_file = f'data/!season_totals/Nuggets/Defense/{file}'
            game_file = os.path.join(subdir, file)
            try:
                # Open the new game's file
                with open(game_file, 'r') as p:
                    game_data = json.load(p)
                
                # open the overall file *if it exists*
                with open(overall_file, 'r') as f:
                    overall_data = json.load(f)
                   
                new_game_data = game_data.copy()
                updated_overall_data = overall_data.copy()
                
                # Format the data into pandas dataframes
                for key in overall_data.keys():
                    updated_overall_data[key]['data'] = pd.DataFrame(overall_data[key]['data']).transpose()
                
                for key in game_data.keys():
                    new_game_data[key]['data'] = pd.DataFrame(game_data[key]['data']).transpose()
                
                # Loop over each shot zone in the new game file and add it to the overall
                for key in new_game_data.keys():
                    if key in updated_overall_data.keys():
                        updated_overall_data[key]['data'] += new_game_data[key]['data']
                        updated_overall_data[key]['shooting_locations'].extend(shot for shot in new_game_data[key]['shooting_locations'])
                        #updated_overall_data[key]['created_locations'].extend(shot for shot in new_game_data[key]['created_locations'])
                    else:
                        updated_overall_data[key] = {'data': new_game_data[key]['data'], 'shooting_locations': new_game_data[key]['shooting_locations']}
                                                     #'created_locations': new_game_data[key]['created_locations']}
                
                # Format back into a JSON
                for key in updated_overall_data.keys():
                    updated_overall_data[key]['data'] = updated_overall_data[key]['data'].to_dict(orient='index')
                
                # Write the new file 
                with open(f'{file}', 'w', encoding='utf-8') as n:
                    json.dump(updated_overall_data, n, ensure_ascii=False, indent=4)
                    
            except:
                print(file, 'No Overall Data')

